In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import spacy


/home/boa/rogan_cluster/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("K-Monitor/kmdb_base")

In [3]:
ds['train'][-1]['text']

'Összesen bruttó 30 milliárd forint értékben javíthat, korszerűsíthet mozdonyokat a MÁV-Start megbízásából a Magyar Vagon Vasúti Járműgyártó, Javító és Karbantartó Kft. - derül ki az uniós közbeszerzési értesítő, a TED pénteki számából.A dokumentum alapján a megrendelő MÁV-cég tavaly decemberben kétrészes keretszerződést kötött a kivitelezővel: bruttó 27,4 milliárd értékben rendelték meg villamos mozdonyok nagyjavítását, korszerűsítését, forgóváz-cseréjét, a dízelmozdonyok esetében pedig bruttó 2,7 milliárd forintot fizetnek a nagygenerálért. Mivel a MÁV-Start csak a megbízási érték 77 százalékára vállalt lehívási kötelezettséget (tehát ennyit biztosan ki kell fizessenek a teljesített munkáért), a végösszeg talán megáll bruttó 23, 2 milliárd forintnál.A MÁV két évet adott a munkák befejezésére, vagyis a kivitelezőnek 2026 decemberéig át kell adnia a kipofozott járműveket. A szerződéskötés tartama érdekes módon rímel Lázár János építési és közlekedési miniszter szerdán ismertetett tízpo

In [4]:
dsf = ds.filter(lambda article: 'Rogán Antal' in article['persons'])

In [5]:
dsf

DatasetDict({
    train: Dataset({
        features: ['news_id', 'title', 'description', 'text', 'source_url', 'archive_url', 'kmdb_url', 'newspaper', 'category', 'pub_time', 'persons', 'institutions', 'places', 'others', 'files'],
        num_rows: 1091
    })
})

In [6]:
texts = [
    f"{row['title']}\n{row['description']}\n\n{row['text']}" 
    for row in dsf['train']
]

In [7]:
print(texts[110])

KÓDOLVA VOLTAK A HIBÁK ROGÁN ÖNKORMÁNYZATÁBAN - ÍRJA A 2013-AS JELENTÉS
Komoly döntési kockázatokkal működött az V. kerületi önkormányzat Rogán Antal polgármestersége idején. Ez egy olyan vizsgálati jelentésből derült ki, amelyet Rogán rendelt meg az után, hogy a kerület ingatlant adott el Portik Tamás élettársának. A jelentés azt kifogásolta, hogy az előterjesztések nem térnek ki minden részletre, így olyan döntések is megszülethetnek, amelyeket egyébként nem biztos, hogy meghoznának.

Komoly döntési kockázatokkal működött az V. kerületi önkormányzat Rogán Antal polgármestersége idején. Ez egy olyan vizsgálati jelentésből derült ki, amelyet Rogán rendelt meg az után, hogy a kerület ingatlant adott el Portik Tamás élettársának. A jelentés azt kifogásolta, hogy az előterjesztések nem térnek ki minden részletre, így olyan döntések is megszülethetnek, amelyeket egyébként nem biztos, hogy meghoznának.
Nyilvánosságra hozta az MSZP azt a hatoldalas jelentést, amelyet Steiner Pál, az V. kerül

In [ ]:
nlp = spacy.load("hu_core_news_md")
nlp.max_length = 10000000

In [9]:
for word in nlp("Írtam egy kisebb szöveget."):
    print(word.text, word.lemma_)



Írtam ír
egy egy
kisebb kis
szöveget szöveg
. .


In [10]:
# Step 1: Load and preprocess the text
def preprocess_text(text):
    # Ensure all Hungarian characters are preserved
    return text

texts = [preprocess_text(text) for text in texts]

In [11]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"  # A lightweight and efficient SentenceBERT model
embedder = SentenceTransformer(model_name)
embeddings = embedder.encode(texts, show_progress_bar=True)

Batches: 100%|██████████| 35/35 [00:05<00:00,  6.36it/s]


In [12]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)


In [13]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


In [14]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech


In [15]:
keybert_model = KeyBERTInspired()


In [16]:
mmr_model = MaximalMarginalRelevance(diversity=0.3)


In [ ]:
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    # "POS": pos_model
}


: 

In [ ]:
# Custom CountVectorizer preserving Hungarian characters
hungarian_stopwords = [
    'és', 'a', 'az', 'hogy', 'nem', 'de', 'mint', 'mert', 'van', 'volt', 'lesz', 'is',
    'egy', 'meg', 'szerint', 'vagy', 'mi', 'azt', 'ez', 'ezt', 'ezek', 'ezeket', 'ennek',
    'még', "miatt", "miért", "mikor", "ki", "új", "nagy", "kis", "kell", "tud", "tudja",
    'már', 'pedig', 'akkor', 'majd', 'minden', 'mindig', 'mindenki', 'együtt', "megint",
    "millió", "milliárd", "rogán", "antal", "személyek", "szervezetek", "helyek", "kulcsszavak", "akták",
    "csak", "el", "több", "kevesebb", "kisebb", "nagyobb", "sem",
]  # Add more Hungarian stopwords as needed

with open("stopwords-hu.txt", "r") as f:
    hungarian_stopwords += f.readlines()

print(f"stopwords: {len(hungarian_stopwords)}")


def custom_preprocessor(text):
    # Optionally, you can normalize text here
    # return text.lower()
    return ' '.join([word.lemma_ for word in nlp(text) if not word.is_stop and not word.is_punct]).lower()


# Custom CountVectorizer preserving Hungarian characters
vectorizer_model = CountVectorizer(
    stop_words=hungarian_stopwords,
    token_pattern=r"(?u)\b\w\w+\b",
    preprocessor=custom_preprocessor,
    min_df=2,
    ngram_range=(1, 2),
)

# Initialize BERTopic with the custom vectorizer
topic_model = BERTopic(
    language="hungarian",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
    embedding_model=embedder,
    # Hyperparameters
    top_n_words=10,
    verbose=True
)

topics, probs = topic_model.fit_transform(texts, embeddings)

2025-01-16 16:37:05,843 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


stopwords: 848


2025-01-16 16:37:14,579 - BERTopic - Dimensionality - Completed ✓
2025-01-16 16:37:14,580 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-16 16:37:14,608 - BERTopic - Cluster - Completed ✓
2025-01-16 16:37:14,610 - BERTopic - Representation - Extracting topics from clusters using representation models.


In [ ]:
# Step 4: Analyze the results
print(topic_model.get_topic_info())  # Get topic information (topic numbers and counts)
print(topic_model.get_topic(0))  # Display the top words for topic 0


    Topic  Count                                              Name  \
0      -1    318                            -1_csak_magyar_el_több   
1       0    180              0_kommunikációs_forint_állami_balásy   
2       1    127        1_önkormányzat_belvárosi_kerületi_ingatlan   
3       2     67          2_letelepedési_magyar_offshore_engedélyt   
4       3     61                          3_orbán_magyar_fidesz_ha   
5       4     59                      4_schadl_völner_varga_magyar   
6       5     45                        5_vizoviczki_sem_el_magyar   
7       6     41                    6_habony_eu line_line_balatoni   
8       7     29  7_miniszterelnöki_kormány_politikai_kabinetiroda   
9       8     24                 8_portik_portik tamás_tamás_hamis   
10      9     24                        9_kertész_schmuck_sem_csak   
11     10     20       10_koncessziós_gyorsforgalmi_mészáros_állam   
12     11     18                          11_fly_fly coop_coop_kft   
13     12     16    

In [ ]:
# Step 5: Save the model and visualize results
topic_model.save("bertopic_model")
visualization = topic_model.visualize_topics()
visualization.show()


2025-01-16 16:30:33,607 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
id_by_topic = {}
filtered_result = topic_model.get_document_info(texts).where(lambda x: x['Probability'] > 0.5).dropna()
for row in filtered_result.iterrows():
    print(row[0], int(row[1]['Topic']))
    id_by_topic.setdefault(int(row[1]['Topic']), []).append(row[0])

0 4
1 5
2 6
4 3
5 1
6 5
7 1
14 3
15 3
16 1
18 3
19 1
20 1
21 3
23 3
26 1
27 17
28 5
29 1
30 5
32 1
33 0
34 3
35 1
36 3
38 1
39 1
41 7
44 5
45 0
47 6
49 1
50 9
52 1
53 1
55 9
56 1
57 9
58 1
59 9
60 9
61 1
62 14
64 3
65 9
67 1
68 14
69 1
71 14
72 1
73 14
75 1
77 6
78 1
80 3
81 3
82 6
83 0
84 1
85 1
86 1
87 1
89 5
90 6
91 5
93 1
97 14
99 1
100 3
101 5
102 1
104 8
105 1
108 14
109 1
111 1
114 14
115 6
116 1
117 5
118 1
120 1
122 1
123 1
125 1
126 1
127 1
128 1
129 1
131 1
134 17
135 14
137 1
138 1
139 1
140 1
141 1
144 1
146 3
148 6
149 0
150 17
151 5
152 1
153 1
155 9
156 3
157 17
158 0
159 1
160 5
161 5
162 17
164 3
165 1
168 17
169 8
170 17
171 6
172 5
173 1
174 1
175 17
176 0
177 5
178 2
179 0
181 3
182 0
183 5
185 0
186 1
189 1
190 3
191 2
192 1
193 7
194 6
195 0
196 7
197 0
199 2
200 0
201 2
203 17
204 0
205 1
209 0
210 1
212 7
213 3
214 7
215 1
216 7
217 0
219 7
220 7
221 3
223 1
224 2
225 14
228 0
229 0
230 0
232 0
233 2
234 0
235 2
236 5
238 0
239 0
241 7
242 14
243 6
244 1
245 0


In [ ]:
id_by_topic

# set topics for ds rows
for topic, ids in id_by_topic.items():
    for id in ids:
        dsf['train'][id]['topic'] = topic

In [ ]:
dsf

DatasetDict({
    train: Dataset({
        features: ['news_id', 'title', 'description', 'text', 'source_url', 'archive_url', 'kmdb_url', 'newspaper', 'category', 'pub_time', 'persons', 'institutions', 'places', 'others', 'files'],
        num_rows: 1091
    })
})

In [ ]:
for topic in sorted(id_by_topic.keys())[2:]:
    print(f"Topic {topic}:")
    print([name for name, score in topic_model.get_topic(topic)])
    for id in id_by_topic[topic]:
        print(f"  {dsf['train'][id]['pub_time']} {dsf['train'][id]['title']}")

Topic 2:
['letelepedési', 'magyar', 'offshore', 'engedélyt', 'cég', 'csak', 'cégek', 'ezer', 'kötvényt', 'külföldi']
  2015-06-09 12:47:00 Letelepedési kötvény: a „jó bevándorlók” pénzt hoznak az offshore-konyhára
  2015-06-08 00:00:00 Ötvenmilliárdos üzlet a letelepedés
  2015-10-13 00:00:00 Ki viszi tovább a letelepedési bizniszt?
  2015-10-17 00:00:00 Kétszázötvenmillió forintért árulták volna az állampolgárságot Fidesz-közeli üzleti körök
  2015-12-01 11:23:00 Offshore milliárdosok csapolják a magyarok pénztárcáját
  2016-01-27 15:25:00 Exim újratöltve
  2016-02-03 14:34:00 Ömlik ki a pénz a külkerbankból, mintha nem lenne holnap
  2016-05-13 15:35:30 Saját cégét is hitelezi bankjával Matolcsy unokatestvére
  2016-07-23 00:00:00 Már azonnal jöhet a pénzes külföldi
  2016-07-26 23:00:00 Habony hitelezőjének testvére felügyeli az Eximbank segélyhiteleit
  2016-08-02 13:43:00 Nem stimmelnek a letelepedési kötvényprogram számai
  2016-07-25 13:32:00 Titkolózik és pénzt kér a hivatal
  